## Data Processing and Filtering

First, we import the excel file that was created in [read_data.ipynb](./read_data.ipynb)

In [7]:
import pandas as pd

file_path = 'data/output/merged.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')  # Specify the sheet name or index

print(len(df))
df.head()

4366


,Title_x,Release Date,Color/B&W,Genre_x,Language_x,Country_x,Rating,Lead Actor,Director Name,Lead Actor FB Likes,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons
0,Over the Hill to the Poorhouse,1920-09-15,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Metropolis,1927-01-26,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Broadway Melody,1929-11-11,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42nd Street,1933-08-29,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Top Hat,1935-04-15,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we combine all **textual** duplicated columns into one. When a row has mulitple values for the duplicated columns, we take the value from the first column (i.e. the first excel file). All the new combined columns are saved into a new dataframe so that the original dataframe is not altered.

In [8]:
new_df = pd.DataFrame()
new_df['Title'] = df['Combined_Title'].combine_first(df['Title'])
new_df['Year'] = df['year'].combine_first(df['Released_Year']).combine_first(df['Year'])
new_df['Director'] = df['Director Name'].combine_first(df['Director_x']).combine_first(df['Director_y'])
new_df['Lead'] = df['Lead Actor'].combine_first(df['Star1'])
new_df['Language'] = df['Language_x'].combine_first(df['Language_y'])
new_df['Country'] = df['Country_x'].combine_first(df['Country_y'])
new_df['Censorship_Rating'] = df['Rating'].combine_first(df['Certificate']).combine_first(df['Rated'])
new_df['Synopsis'] = df['Overview'].combine_first(df['Plot'])

Next, we combine duplicated columns that have **lists** which in our case is the cast and genre. After combining the lists we take out potential duplicates within the lists as well while retaining all unique values.

In [9]:
cast_x = df[['Star1', 'Star2', 'Star3', 'Star4']].fillna('').values.tolist()
cast_y = df['Actors'].fillna('').apply(lambda x: x.split(', ')).tolist()
df_combined = pd.DataFrame({'cast_x': cast_x, 'cast_y': cast_y})
new_df['Cast'] = df_combined.apply(
    lambda row: ', '.join(filter(lambda x: x != '', set(row['cast_x'] + row['cast_y']))), axis=1
)

df_combined = pd.DataFrame({'Genre_x': df['Genre_x'].fillna(''), 'Genre_y': df['Genre_y'].fillna(''), 'Genre': df['Genre'].fillna('')})
new_df['Genre'] = df_combined.apply(
    lambda row: ', '.join(sorted(set(filter(lambda x: x != '', 
                                 row['Genre_x'].split(', ') + 
                                 row['Genre_y'].split(', ') + 
                                 row['Genre'].split(', '))))),
                      axis=1
)

Then we combine **numerical** duplicated columns by taking the average of the columns.

In [10]:
new_df['Runtime'] = (
    df['Duration (min)'].fillna(0) + 
    df['Runtime_x'].str.extract('(\d+)')[0].fillna(0).astype(int) + 
    df['Runtime_y'].str.extract('(\d+)')[0].fillna(0).astype(int)
) / 3

new_df['Revenue'] = (
    df['Gross Revenue'].fillna(0) + 
    df['Gross'].str.extract('(\d+)')[0].fillna(0).astype(int) + 
    df['BoxOffice'].str.extract('(\d+)')[0].fillna(0).astype(int)
) / 3

new_df['Votes'] = (
    df['No_of_Votes'].fillna(0) + 
    df['imdbVotes'].str.extract('(\d+)')[0].fillna(0).astype(int)
) / 2

new_df['IMDB rating'] = (
    df['IMDb Score (1-10)'].fillna(0) + 
    df['imdbRating'].fillna(0)
) / 2

new_df['Metascore'] = (
    df['Meta_score'].fillna(0) + 
    df['Metascore'].fillna(0)
) / 2

Finally, we export the new dataframe with all the desired information into an excel file called [merged2.xlsx](./data/output/merged2.xlsx)

In [13]:
new_df.to_excel('data/output/merged2.xlsx', index=False)